In [2]:
import sys
print(sys.version)

3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]


In [1]:
import cv2
import mediapipe as mp

print("OpenCV version:", cv2.__version__)
print("MediaPipe imported successfully!")

OpenCV version: 4.11.0
MediaPipe imported successfully!


In [3]:
import cv2

cap = cv2.VideoCapture(0)  # 웹캠 사용
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
else:
    print("웹캠 정상 작동")
cap.release()

웹캠 정상 작동


In [10]:
import cv2
import mediapipe as mp

# sample 실행
# MediaPipe Pose 모듈 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,  # 동영상(연속 프레임) 처리에 적합
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 영상 파일 경로 설정 (백슬래시를 raw string으로 처리)
video_path = r"C:\barunrun\sample_video\sample_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("영상 파일을 열 수 없습니다. 경로를 확인해 주세요.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 영상의 끝에 도달하면 종료

    # BGR 이미지를 RGB로 변환 (MediaPipe는 RGB 이미지를 사용)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # 인식된 랜드마크가 있다면 영상에 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )

    cv2.imshow("MediaPipe Pose", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [22]:
import cv2
import mediapipe as mp
import numpy as np

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

video_path = r"C:\barunrun\sample_video\sample_video2.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("영상 파일을 열 수 없습니다. 경로를 확인해 주세요.")
    exit()

# 윈도우를 정상 모드로 생성 후 크기 조절 (예: 800x600)
window_name = "바른런 서비스"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 900)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        landmarks = results.pose_landmarks.landmark
        left_hip = [landmarks[23].x, landmarks[23].y]
        left_knee = [landmarks[25].x, landmarks[25].y]
        left_ankle = [landmarks[27].x, landmarks[27].y]
        
        h, w, _ = frame.shape
        left_hip_px = (int(left_hip[0] * w), int(left_hip[1] * h))
        left_knee_px = (int(left_knee[0] * w), int(left_knee[1] * h))
        left_ankle_px = (int(left_ankle[0] * w), int(left_ankle[1] * h))
        
        angle = calculate_angle(left_hip_px, left_knee_px, left_ankle_px)
        
        cv2.putText(frame, str(int(angle)), left_knee_px, 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        if angle < 160:
            cv2.putText(frame, "자세 교정 필요", (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow(window_name, frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [23]:
import cv2
import mediapipe as mp
import numpy as np

def calculate_angle(a, b, c):
    """
    세 점 (각각 [x, y] 좌표) 사이의 각도를 계산하는 함수입니다.
    중간 점 b에서 이루어지는 각도를 반환합니다.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# 각 부위별 임계값 설정 (필요에 따라 조정 가능)
thresholds = {
    'left_knee': 160,   # 왼쪽 무릎의 임계 각도 (예: 160도)
    # 예시: 'right_knee': 160, 'left_elbow': 170 등 추가 가능
}

# 각 부위별 피드백 메시지 설정
feedback_messages = {
    'left_knee': "Left knee angle is too low. Please straighten your leg.",
    # 예시: 'right_knee': "Right knee angle is too low.", 'left_elbow': "Left elbow angle is abnormal." 등
}

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,  # 동영상(연속 프레임) 처리를 위해 False로 설정
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 영상 파일 경로 설정 (raw string 사용)
video_path = r"C:\barunrun\sample_video\sample_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("영상 파일을 열 수 없습니다. 경로를 확인해 주세요.")
    exit()

# 창을 생성하고 크기를 800x600 픽셀로 조절
window_name = "BarunRun Service"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 900)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 영상의 끝에 도달하면 루프 종료

    # BGR 이미지를 RGB로 변환 (MediaPipe는 RGB 이미지를 사용)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # 프레임에 자세 랜드마크를 그림
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # 랜드마크 좌표 추출
        landmarks = results.pose_landmarks.landmark
        
        # 왼쪽 무릎 각도 계산 (왼쪽 엉덩이: 인덱스 23, 왼쪽 무릎: 인덱스 25, 왼쪽 발목: 인덱스 27)
        left_hip = [landmarks[23].x, landmarks[23].y]
        left_knee = [landmarks[25].x, landmarks[25].y]
        left_ankle = [landmarks[27].x, landmarks[27].y]
        
        # 영상의 실제 크기에 맞게 좌표 변환
        h, w, _ = frame.shape
        left_hip_px = (int(left_hip[0] * w), int(left_hip[1] * h))
        left_knee_px = (int(left_knee[0] * w), int(left_knee[1] * h))
        left_ankle_px = (int(left_ankle[0] * w), int(left_ankle[1] * h))
        
        # 왼쪽 무릎 각도 계산
        angle_left_knee = calculate_angle(left_hip_px, left_knee_px, left_ankle_px)
        
        # 무릎 근처에 각도 출력 (정수형으로 변환)
        cv2.putText(frame, str(int(angle_left_knee)), left_knee_px, 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # 왼쪽 무릎 각도가 임계값보다 낮으면 피드백 메시지 출력
        if angle_left_knee < thresholds['left_knee']:
            cv2.putText(frame, feedback_messages['left_knee'], (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow(window_name, frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

def calculate_angle(a, b, c):
    """
    세 점 (각각 [x, y] 좌표) 사이의 각도를 계산하는 함수입니다.
    중간 점 b에서 이루어지는 각도를 반환합니다.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# 각 부위별 임계값 설정 (필요에 따라 조정 가능)
thresholds = {
    'left_knee': 160,    # 왼쪽 무릎 임계 각도
    'right_knee': 160,   # 오른쪽 무릎 임계 각도
    'left_hip': 140,     # 왼쪽 엉덩이(hip) 임계 각도
    'right_hip': 140,    # 오른쪽 엉덩이(hip) 임계 각도
}

# 각 부위별 피드백 메시지 설정
feedback_messages = {
    'left_knee': "Left knee angle too low. Straighten your leg.",
    'right_knee': "Right knee angle too low. Straighten your leg.",
    'left_hip': "Left hip too bent. Keep your torso upright.",
    'right_hip': "Right hip too bent. Keep your torso upright.",
}

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,  # 동영상(연속 프레임) 처리를 위해 False로 설정
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 영상 파일 경로 설정 (raw string 사용)
video_path = r"C:\barunrun\sample_video\sample_video.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("영상 파일을 열 수 없습니다. 경로를 확인해 주세요.")
    exit()

# 창을 생성하고 크기를 800x900 픽셀로 조절
window_name = "BarunRun Service"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 900)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 영상의 끝에 도달하면 루프 종료

    # BGR 이미지를 RGB로 변환 (MediaPipe는 RGB 이미지를 사용)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # 프레임에 자세 랜드마크를 그림
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmarks = results.pose_landmarks.landmark
        h, w, _ = frame.shape

        # 왼쪽 무릎 각도 계산 (왼쪽 엉덩이: 인덱스 23, 왼쪽 무릎: 인덱스 25, 왼쪽 발목: 인덱스 27)
        left_hip = (int(landmarks[23].x * w), int(landmarks[23].y * h))
        left_knee = (int(landmarks[25].x * w), int(landmarks[25].y * h))
        left_ankle = (int(landmarks[27].x * w), int(landmarks[27].y * h))
        angle_left_knee = calculate_angle(left_hip, left_knee, left_ankle)

        # 오른쪽 무릎 각도 계산 (오른쪽 엉덩이: 인덱스 24, 오른쪽 무릎: 인덱스 26, 오른쪽 발목: 인덱스 28)
        right_hip = (int(landmarks[24].x * w), int(landmarks[24].y * h))
        right_knee = (int(landmarks[26].x * w), int(landmarks[26].y * h))
        right_ankle = (int(landmarks[28].x * w), int(landmarks[28].y * h))
        angle_right_knee = calculate_angle(right_hip, right_knee, right_ankle)

        # 왼쪽 엉덩이 각도 계산 (왼쪽 어깨: 인덱스 11, 왼쪽 엉덩이: 인덱스 23, 왼쪽 무릎: 인덱스 25)
        left_shoulder = (int(landmarks[11].x * w), int(landmarks[11].y * h))
        angle_left_hip = calculate_angle(left_shoulder, left_hip, left_knee)

        # 오른쪽 엉덩이 각도 계산 (오른쪽 어깨: 인덱스 12, 오른쪽 엉덩이: 인덱스 24, 오른쪽 무릎: 인덱스 26)
        right_shoulder = (int(landmarks[12].x * w), int(landmarks[12].y * h))
        angle_right_hip = calculate_angle(right_shoulder, right_hip, right_knee)

        # 각도 정보 화면에 출력 (부위별 간략 표시)
        cv2.putText(frame, f"LK: {int(angle_left_knee)}", left_knee, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"RK: {int(angle_right_knee)}", right_knee, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"LH: {int(angle_left_hip)}", left_hip, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"RH: {int(angle_right_hip)}", right_hip, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

        # 피드백 메시지 출력 (화면 좌측 상단 영역에 순차적으로 출력)
        y_offset = 50  # 피드백 메시지 시작 y 좌표
        if angle_left_knee < thresholds['left_knee']:
            cv2.putText(frame, feedback_messages['left_knee'], (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
            y_offset += 30
        if angle_right_knee < thresholds['right_knee']:
            cv2.putText(frame, feedback_messages['right_knee'], (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
            y_offset += 30
        if angle_left_hip < thresholds['left_hip']:
            cv2.putText(frame, feedback_messages['left_hip'], (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
            y_offset += 30
        if angle_right_hip < thresholds['right_hip']:
            cv2.putText(frame, feedback_messages['right_hip'], (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
            y_offset += 30

    cv2.imshow(window_name, frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [27]:
import cv2
import mediapipe as mp
import numpy as np

def calculate_angle(a, b, c):
    """
    세 점 (각각 [x, y] 좌표) 사이의 각도를 계산하는 함수입니다.
    중간 점 b에서 이루어지는 각도를 반환합니다.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# 각 부위별 임계값 설정 (필요에 따라 조정 가능)
thresholds = {
    'left_knee': 160,
    'right_knee': 160,
    'left_hip': 140,
    'right_hip': 140,
}

# 각 부위별 피드백 메시지 설정
feedback_messages = {
    'left_knee': "Left knee angle too low.",
    'right_knee': "Right knee angle too low.",
    'left_hip': "Left hip too bent.",
    'right_hip': "Right hip too bent.",
}

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,  # 동영상(연속 프레임) 처리를 위해 False로 설정
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 영상 파일 경로 설정 (raw string 사용)
video_path = r"C:\barunrun\sample_video\sample_video.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("영상 파일을 열 수 없습니다. 경로를 확인해 주세요.")
    exit()

# 창을 생성하고 크기를 800x900 픽셀로 조절
window_name = "BarunRun Service"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 900)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 영상의 끝에 도달하면 루프 종료

    # BGR 이미지를 RGB로 변환 (MediaPipe는 RGB 이미지를 사용)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # 프레임에 자세 랜드마크를 그림
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        landmarks = results.pose_landmarks.landmark
        h, w, _ = frame.shape

        # -----------------------------
        # 1) 왼쪽 무릎 각도 계산
        left_hip = (int(landmarks[23].x * w), int(landmarks[23].y * h))
        left_knee = (int(landmarks[25].x * w), int(landmarks[25].y * h))
        left_ankle = (int(landmarks[27].x * w), int(landmarks[27].y * h))
        angle_left_knee = calculate_angle(left_hip, left_knee, left_ankle)
        
        # 2) 오른쪽 무릎 각도 계산
        right_hip = (int(landmarks[24].x * w), int(landmarks[24].y * h))
        right_knee = (int(landmarks[26].x * w), int(landmarks[26].y * h))
        right_ankle = (int(landmarks[28].x * w), int(landmarks[28].y * h))
        angle_right_knee = calculate_angle(right_hip, right_knee, right_ankle)

        # 3) 왼쪽 엉덩이 각도 계산
        left_shoulder = (int(landmarks[11].x * w), int(landmarks[11].y * h))
        angle_left_hip = calculate_angle(left_shoulder, left_hip, left_knee)

        # 4) 오른쪽 엉덩이 각도 계산
        right_shoulder = (int(landmarks[12].x * w), int(landmarks[12].y * h))
        angle_right_hip = calculate_angle(right_shoulder, right_hip, right_knee)

        # -----------------------------
        # 각도 정보 화면에 출력 (부위별 간략 표시)
        cv2.putText(frame, f"LK: {int(angle_left_knee)}", left_knee, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"RK: {int(angle_right_knee)}", right_knee, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"LH: {int(angle_left_hip)}", left_hip, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"RH: {int(angle_right_hip)}", right_hip, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

        # -----------------------------
        # 피드백 메시지 출력 (화면 좌측 상단 영역에 순차적으로 출력)
        y_offset = 50  # 피드백 메시지 시작 y 좌표
        
        # 각 관절을 순회하며, 임계값보다 작으면 백분율로 얼마나 부족한지 계산
        angles = {
            'left_knee': angle_left_knee,
            'right_knee': angle_right_knee,
            'left_hip': angle_left_hip,
            'right_hip': angle_right_hip
        }
        
        for joint_name, angle_value in angles.items():
            threshold_value = thresholds[joint_name]
            if angle_value < threshold_value:
                # 부족한 각도 차이
                diff = threshold_value - angle_value
                # (diff / threshold_value) * 100 => 임계값 대비 부족한 백분율
                percent_diff = (diff / threshold_value) * 100
                message = (f"{feedback_messages[joint_name]} "
                           f"({percent_diff:.1f}% below recommended)")
                
                cv2.putText(frame, message, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                            0.8, (0, 0, 255), 2)
                y_offset += 30

    cv2.imshow(window_name, frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
